-Merging DataFrames

-Pandas Idioms

-Groupby

-Scales

-Pivot Table

-Date/Time Functionality

# Merging Dataframes

Nessa seção vamos ver como juntar dataframes, seja juntando eles horizontalmente(merge) ou verticalmente(concate).

Antes de prosseguirmos, será útil revisar algumas definições de teoria de grupos.

![Venn Diagram](merging1.png)

Isso é um Diagrama de Venn.
O círculo da esquerda representa os estudantes de uma Universidade.
O círculo da direita representa os funcionários da Universidade.
A região de superposição representa aqueles que são estudantes e funcionários da Universidade.


No caso do Pandas, podemos pensar no caso em que podemos ter essas duas populações como índices em DataFrames separados, talvez o Nome da pessoa, por exemplo.

Quando queremos juntar esses DataFrames, nós temos algumas escolhas a fazer.

Primeiro, se quisermos essas pessoas independente se elas são estudantes ou funcionários.Então queremos fazer uma União em teoria de grupos, ou uma 'full outer join' na nomenclatura de dataBase.

![Union](merging2.png)

Se quisermos aqueles que são estudantes E funcionários.
Então queremos fazer uma Intersecção em teoria de grupos, ou uma 'inner join' na nomenclatura de dataBase.

![Intersection](merging3.png)

In [5]:
# Vamos ver um exemplo

import pandas as pd

#Primeiro vamos criar os DataFrames 'students' e 'staff'
staff_df=pd.DataFrame([{'Name':'Kelly','Role':'Director of HR'},
                      {'Name':'Sally','Role':'Course liasion'},
                      {'Name':'James','Role':'Grader'}])

student_df=pd.DataFrame([{'Name':'James','School':'Business'},
                         {'Name':'Mike','School':'Law'},
                          {'Name':'Sally','School':'Engineering'}])
print(staff_df)
print(student_df)

    Name            Role
0  Kelly  Director of HR
1  Sally  Course liasion
2  James          Grader
    Name       School
0  James     Business
1   Mike          Law
2  Sally  Engineering


# Merge

In [6]:
# IMPORTANTE: Os DataFrames devem ser indexados na coluna que queremos juntar, como queremos juntar os dataFrames por nome,
#vamos setar a coluna 'Name' como indice em ambos os Dataframes.
staff_df.set_index('Name',inplace=True)
student_df.set_index('Name',inplace=True)

In [3]:
# Podemos ver que a Kelly é só estudante, o Mike é só funcionário e o James e a Sally são estudantes e funcionários.

### Union / outer

In [8]:
# Se quisermos unir esses DataFrames, nós chamamos a função merge() passando como parâmetros os DataFrames que queremos juntar,
# e modo(nessa caso 'outer'). Queremos usar os indices esquerdo e direito como colunas de 'junção'.

pd.merge(staff_df, student_df, how='outer', left_index=True, right_index=True)

,Role,School
Name,,
James,Grader,Business
Kelly,Director of HR,NaN
Mike,NaN,Law
Sally,Course liasion,Engineering


In [5]:
# Podemos ver que todos os nomes estão sendo listados no DataFrame resultante.
# E como Mike não tem um emprego na Universidade e Kelly não estuda na Universidade, as células são listadas como 'missing values'.

### Intersection / inner

In [6]:
# Se quisermos pegar a intersecção, ou seja, aqueles que são funcionários e estudam na Universidade, utilizamos o atributo 'inner'.
pd.merge(staff_df,student_df,how='inner', left_index=True, right_index=True)

,Role,School
Name,,
Sally,Course liasion,Engineering
James,Grader,Business


In [7]:
# Podemos ver que o resultado só tem aqueles que trabalham e estudam na Universidade.

### Set Addition

Existe outros dois casos quando queremos juntar DataFrames.

-Podemos querer pegar todas as pessoas que são funcionários, independentes se são estudantes ou não.
Se forem estudantes, vamos querer pegar essas informaçoes também. É o que é chamado de **left join**. O DataFrame da esquerda será preservado e o DataFrame da direita se juntará a ele.

-Podemos querer pegar todas as pessoas que são estudantes, independentes se são funcionários ou não.
Se forem funcionários, vamos querer pegar essas informaçoes também. É o que é chamado de **right join**. O DataFrame da direita será preservado e o DataFrame da esquerda se juntará a ele.

In [8]:
pd.merge(staff_df, student_df, how='left', left_index=True, right_index=True)

,Role,School
Name,,
Kelly,Director of HR,NaN
Sally,Course liasion,Engineering
James,Grader,Business


In [9]:
pd.merge(staff_df, student_df, how='right', left_index=True, right_index=True)

,Role,School
Name,,
James,Grader,Business
Mike,NaN,Law
Sally,Course liasion,Engineering


### 'on' Parameter

In [10]:
# Podemos fazer de outro jeito, não precisamos usar os parâmetros 'left_index' e 'right_index' indicando que queremos juntas as colunas.
# Podemos utilizar o parâmetro 'on' indicando a coluna que queremos mesclar.

In [11]:
# Nesse caso não precisamos utilizar a coluna 'Name' como índice.Nesse caso a diferença entre usar e não usar será no index do
# DataFrame retornado.
staff_df = staff_df.reset_index()
student_df = student_df.reset_index()

In [12]:
pd.merge(staff_df, student_df, how='outer', on='Name')

,Name,Role,School
0,Kelly,Director of HR,NaN
1,Sally,Course liasion,Engineering
2,James,Grader,Business
3,Mike,NaN,Law


In [13]:
pd.merge(staff_df, student_df, how='inner', on='Name')

,Name,Role,School
0,Sally,Course liasion,Engineering
1,James,Grader,Business


In [14]:
pd.merge(staff_df, student_df, how='right', on='Name')

,Name,Role,School
0,James,Grader,Business
1,Mike,NaN,Law
2,Sally,Course liasion,Engineering


In [15]:
pd.merge(staff_df, student_df, how='left', on='Name')

,Name,Role,School
0,Kelly,Director of HR,NaN
1,Sally,Course liasion,Engineering
2,James,Grader,Business


### Case that there is conflicts between the DataFrames

In [16]:
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR', 
                          'Location': 'State Street'},
                         {'Name': 'Sally', 'Role': 'Course liasion', 
                          'Location': 'Washington Avenue'},
                         {'Name': 'James', 'Role': 'Grader', 
                          'Location': 'Washington Avenue'}])
student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business', 
                            'Location': '1024 Billiard Avenue'},
                           {'Name': 'Mike', 'School': 'Law', 
                            'Location': 'Fraternity House #22'},
                           {'Name': 'Sally', 'School': 'Engineering', 
                            'Location': '512 Wilson Crescent'}])

In [17]:
# Ambos os DataFrames possuem as colunas 'Name' e 'Location'.
# Mas, podemos ver que a coluna Location é diferente para a mesma pessoa. Isso se deve pelo fato de que a coluna 'Location' no DataFrame
# 'staff_df' se refere ao local do trabalho, enquanto no DataFrame 'student_df' se refere ao local que o estudante mora.
# Acontece que a função merge() preserva essa informação mas adiciona '_x' (representando o DataFrame da esquerda) e '_y'(representando o DataFrame da direita)
# à coluna em conflito.

# Exemplo, se quisermos as informações só dos funcionários
pd.merge(staff_df, student_df, how='left', on='Name')

# 'Location_x' representa a coluna 'Location' do DataFrame 'staff_d' e 'Location_y' representa a coluna 'Location' do Dataframe 'student_df'.

,Name,Role,Location_x,School,Location_y
0,Kelly,Director of HR,State Street,NaN,NaN
1,Sally,Course liasion,Washington Avenue,Engineering,512 Wilson Crescent
2,James,Grader,Washington Avenue,Business,1024 Billiard Avenue


### Multi-indexing and multiple columns

In [18]:
# Vamos supor que tenhamos dois DataFrames com as colunas 'First Name' e 'Last Name'.
# Pode ser que o primeiro nome dos funcionários e estudantes venham a coincidir, mas o último nome não.

# Nesse caso, utilizamos uma lista no parâmetro 'on' indicando múltiplas colunas para fazer a mesclagem.

# Exemplo:
staff_df = pd.DataFrame([{'First Name': 'Kelly', 'Last Name': 'Desjardins', 
                          'Role': 'Director of HR'},
                         {'First Name': 'Sally', 'Last Name': 'Brooks', 
                          'Role': 'Course liasion'},
                         {'First Name': 'James', 'Last Name': 'Wilde', 
                          'Role': 'Grader'}])
student_df = pd.DataFrame([{'First Name': 'James', 'Last Name': 'Hammond', 
                            'School': 'Business'},
                           {'First Name': 'Mike', 'Last Name': 'Smith', 
                            'School': 'Law'},
                           {'First Name': 'Sally', 'Last Name': 'Brooks', 
                            'School': 'Engineering'}])


In [19]:
# Podemos ver que existe duas pessoas com o nome 'James', mas sobrenomes diferentes.
# Então, esperamos que a intersecção não pegue essas duas pessoas.
pd.merge(staff_df, student_df, how='inner', on=['First Name','Last Name'])

,First Name,Last Name,Role,School
0,Sally,Brooks,Course liasion,Engineering


# Concate

Juntar os DataFrames verticalmente.

In [20]:
# Exemplo: Vamos supor que tenhamos algum tipo de  informação ao longo de alguns anos e que cada arquivo tenha a informação apenas de um ano.

# Esses DataFrames estão bagunçados, então iremos evitar as mensages de aviso do jupyter utilizando '%%capture' e utilizar o
# parâmetro 'error_bad_lines' para ignorar as linhas que causam problema.

In [9]:
%%capture
df_2011 = pd.read_csv("datasets/college_scorecard/MERGED2011_12_PP.csv", error_bad_lines=False)
df_2012 = pd.read_csv("datasets/college_scorecard/MERGED2012_13_PP.csv", error_bad_lines=False)
df_2013 = pd.read_csv("datasets/college_scorecard/MERGED2013_14_PP.csv", error_bad_lines=False)

In [22]:
df_2011.head(3)

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,OMAWDP8_NOTFIRSTTIME_POOLED_SUPP,OMENRUP_NOTFIRSTTIME_POOLED_SUPP,OMENRYP_FULLTIME_POOLED_SUPP,OMENRAP_FULLTIME_POOLED_SUPP,OMAWDP8_FULLTIME_POOLED_SUPP,OMENRUP_FULLTIME_POOLED_SUPP,OMENRYP_PARTTIME_POOLED_SUPP,OMENRAP_PARTTIME_POOLED_SUPP,OMAWDP8_PARTTIME_POOLED_SUPP,OMENRUP_PARTTIME_POOLED_SUPP
0,100654.0,100200.0,1002,Alabama A & M University,Normal,AL,35762,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100663.0,105200.0,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100690.0,2503400.0,25034,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Vamos juntar esses DataFrames
pd.concat([df_2011,df_2012,df_2013])

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,OMAWDP8_NOTFIRSTTIME_POOLED_SUPP,OMENRUP_NOTFIRSTTIME_POOLED_SUPP,OMENRYP_FULLTIME_POOLED_SUPP,OMENRAP_FULLTIME_POOLED_SUPP,OMAWDP8_FULLTIME_POOLED_SUPP,OMENRUP_FULLTIME_POOLED_SUPP,OMENRYP_PARTTIME_POOLED_SUPP,OMENRAP_PARTTIME_POOLED_SUPP,OMAWDP8_PARTTIME_POOLED_SUPP,OMENRUP_PARTTIME_POOLED_SUPP
0,100654.0,100200.0,1002,Alabama A & M University,Normal,AL,35762,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100663.0,105200.0,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100690.0,2503400.0,25034,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100706.0,105500.0,1055,University of Alabama in Huntsville,Huntsville,AL,35899,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100724.0,100500.0,1005,Alabama State University,Montgomery,AL,36104-0271,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7799,48285703.0,157107.0,1571,Georgia Military College-Columbus Campus,Columbus,GA,31909,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7800,48285704.0,157101.0,1571,Georgia Military College-Valdosta Campus,Valdosta,GA,31605,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7801,48285705.0,157105.0,1571,Georgia Military College-Warner Robins Campus,Warner Robins,GA,31093,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7802,48285706.0,157100.0,1571,Georgia Military College-Online,Milledgeville,GA,31061,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# Podemos ver que existem 30832 linhas nesse DataFrame, o que corresponde a soma dos 3 DataFrames:

len(df_2011)+len(df_2012)+len(df_2013)

30832

In [13]:
# Mas temos um problema, como todos os dados estão concatenados, não sabemos qual é de que ano.
# Podemos utilizar o parâmetro 'keys' para sanar esse problema.

df_result=pd.concat([df_2011,df_2012,df_2013], keys=['2011','2012','2013'])
df_result

UNITID      OPEID OPEID6  \
2011 0       100654.0   100200.0   1002   
     1       100663.0   105200.0   1052   
     2       100690.0  2503400.0  25034   
     3       100706.0   105500.0   1055   
     4       100724.0   100500.0   1005   
...               ...        ...    ...   
2013 7799  48285703.0   157107.0   1571   
     7800  48285704.0   157101.0   1571   
     7801  48285705.0   157105.0   1571   
     7802  48285706.0   157100.0   1571   
     7803  48285707.0   157103.0   1571   

                                                  INSTNM            CITY  \
2011 0                          Alabama A & M University          Normal   
     1               University of Alabama at Birmingham      Birmingham   
     2                                Amridge University      Montgomery   
     3               University of Alabama in Huntsville      Huntsville   
     4                          Alabama State University      Montgomery   
...                                                  ...             ...   
2013 7799       Georgia Military College-Columbus Campus        Columbus   
     7800       Georgia Military College-Valdosta Campus        Valdosta   
     7801  Georgia Military College-Warner Robins Campus   Warner Robins   
     7802                Georgia Military College-Online   Milledgeville   
     7803        Georgia Military College-Stone Mountain  Stone Mountain   

          STABBR         ZIP  ACCREDAGENCY INSTURL NPCURL  ...  \
2011 0        AL       35762           NaN     NaN    NaN  ...   
     1        AL  35294-0110           NaN     NaN    NaN  ...   
     2        AL  36117-3553           NaN     NaN    NaN  ...   
     3        AL       35899           NaN     NaN    NaN  ...   
     4        AL  36104-0271           NaN     NaN    NaN  ...   
...          ...         ...           ...     ...    ...  ...   
2013 7799     GA       31909           NaN     NaN    NaN  ...   
     7800     GA       31605           NaN     NaN    NaN  ...   
     7801     GA       31093           NaN     NaN    NaN  ...   
     7802     GA       31061           NaN     NaN    NaN  ...   
     7803     GA       30083           NaN     NaN    NaN  ...   

          OMAWDP8_NOTFIRSTTIME_POOLED_SUPP OMENRUP_NOTFIRSTTIME_POOLED_SUPP  \
2011 0                                 NaN                              NaN   
     1                                 NaN                              NaN   
     2                                 NaN                              NaN   
     3                                 NaN                              NaN   
     4                                 NaN                              NaN   
...                                    ...                              ...   
2013 7799                              NaN                              NaN   
     7800                              NaN                              NaN   
     7801                              NaN                              NaN   
     7802                              NaN                              NaN   
     7803                              NaN                              NaN   

          OMENRYP_FULLTIME_POOLED_SUPP OMENRAP_FULLTIME_POOLED_SUPP  \
2011 0                             NaN                          NaN   
     1                             NaN                          NaN   
     2                             NaN                          NaN   
     3                             NaN                          NaN   
     4                             NaN                          NaN   
...                                ...                          ...   
2013 7799                          NaN                          NaN   
     7800                          NaN                          NaN   
     7801                          NaN                          NaN   
     7802                          NaN                          NaN   
     7803                          NaN                        

In [26]:
df_result.index

MultiIndex([('2011',    0),
            ('2011',    1),
            ('2011',    2),
            ('2011',    3),
            ('2011',    4),
            ('2011',    5),
            ('2011',    6),
            ('2011',    7),
            ('2011',    8),
            ('2011',    9),
            ...
            ('2013', 7794),
            ('2013', 7795),
            ('2013', 7796),
            ('2013', 7797),
            ('2013', 7798),
            ('2013', 7799),
            ('2013', 7800),
            ('2013', 7801),
            ('2013', 7802),
            ('2013', 7803)],
           length=30832)

In [27]:
# Importante: A função concat() também possui os métodos 'inner' e 'outer'. Se quisermos juntar DataFrames que não possuem colunas idênticas.


# Pandas Idioms

Existem diferentes modos de solucionar um problema, mas uns são mais apropriados que outros. Os melhores jeitos de resolver 

são chamados de **Idiomatic Python**.

No caso do Pandas, chamamos esses jeitos de **pandorable**.

In [28]:
import pandas as pd
import numpy as np
import timeit

df=pd.read_csv("datasets/census.csv")
df.head()


,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


### chaining

In [29]:
# O primeiro dos "Pandas idioms" é o método "chaining". A ideia geral é que podemos ir 'acorrentando' diferentes operações em um DataFrame.

# Aqui está um exemplo de um jeito 'pandorable' de escrever o código utilizando o método 'chaining'.
# Nessa código queremos excluir as linhas que apresentam 'SUMLEV' diferente de 50,  colocar os indices 'STNAME' e 'CTYNAME' 
# e renomear a coluna 'ESTIMATESBASE2010'.

In [30]:
(df.where(df['SUMLEV']==50)
    
    .dropna()
    .set_index(['STNAME','CTYNAME'])
    .rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'}))

SUMLEV  REGION  DIVISION  STATE  COUNTY  \
STNAME  CTYNAME                                                      
Alabama Autauga County       50.0     3.0       6.0    1.0     1.0   
        Baldwin County       50.0     3.0       6.0    1.0     3.0   
        Barbour County       50.0     3.0       6.0    1.0     5.0   
        Bibb County          50.0     3.0       6.0    1.0     7.0   
        Blount County        50.0     3.0       6.0    1.0     9.0   
...                           ...     ...       ...    ...     ...   
Wyoming Sweetwater County    50.0     4.0       8.0   56.0    37.0   
        Teton County         50.0     4.0       8.0   56.0    39.0   
        Uinta County         50.0     4.0       8.0   56.0    41.0   
        Washakie County      50.0     4.0       8.0   56.0    43.0   
        Weston County        50.0     4.0       8.0   56.0    45.0   

                           CENSUS2010POP  Estimates Base 2010  \
STNAME  CTYNAME                                                 
Alabama Autauga County           54571.0              54571.0   
        Baldwin County          182265.0             182265.0   
        Barbour County           27457.0              27457.0   
        Bibb County              22915.0              22919.0   
        Blount County            57322.0              57322.0   
...                                  ...                  ...   
Wyoming Sweetwater County        43806.0              43806.0   
        Teton County             21294.0              21294.0   
        Uinta County             21118.0              21118.0   
        Washakie County           8533.0               8533.0   
        Weston County             7208.0               7208.0   

                           POPESTIMATE2010  POPESTIMATE2011  POPESTIMATE2012  \
STNAME  CTYNAME                                                                
Alabama Autauga County             54660.0          55253.0          55175.0   
        Baldwin County            183193.0         186659.0         190396.0   
        Barbour County             27341.0          27226.0          27159.0   
        Bibb County                22861.0          22733.0          22642.0   
        Blount County              57373.0          57711.0          57776.0   
...                                    ...              ...              ...   
Wyoming Sweetwater County          43593.0          44041.0          45104.0   
        Teton County               21297.0          21482.0          21697.0   
        Uinta County               21102.0          20912.0          20989.0   
        Washakie County             8545.0           8469.0           8443.0   
        Weston County               7181.0           7114.0           7065.0   

                           ...  RDOMESTICMIG2011  RDOMESTICMIG2012  \
STNAME  CTYNAME            ...                                       
Alabama Autauga County     ...          7.242091         -2.915927   
        Baldwin County     ...         14.832960         17.647293   
        Barbour County     ...         -4.728132         -2.500690   
        Bibb County        ...         -5.527043         -5.068871   
        Blount County      ...          1.807375         -1.177622   
...                        ...               ...               ...   
Wyoming Sweetwater County  ...          1.072643         16.243199   
        Teton County       ...         -1.589565          0.972695   
        Uinta County       ...        -17.755986         -4.916350   
        Washakie County    ...        -11.637475         -0.827815   
        Weston County      ...        -11.752361         -8.040059   

                           RDOMESTICMIG2013  RDOMESTICMIG2014  \
STNAME  CTYNAME                                                 
Alabama Autauga County            -3.012349          2.265971   
        Baldwin County            21.845705         19.243287   
        Barbour County            -7.056824         -3.904

In [31]:
# Aqui um jeito não 'pandorable' de fazer a mesma coisa que a célula anterior.

# Primeiro criamos um dataframe a partir do original.
df =df[df['SUMLEV'] ==50 ]

#Depois setamos as novas colunas
df.set_index(['STNAME','CTYNAME'], inplace=True)

# E então renomeamos a coluna
df.rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'})

SUMLEV  REGION  DIVISION  STATE  COUNTY  \
STNAME  CTYNAME                                                      
Alabama Autauga County         50       3         6      1       1   
        Baldwin County         50       3         6      1       3   
        Barbour County         50       3         6      1       5   
        Bibb County            50       3         6      1       7   
        Blount County          50       3         6      1       9   
...                           ...     ...       ...    ...     ...   
Wyoming Sweetwater County      50       4         8     56      37   
        Teton County           50       4         8     56      39   
        Uinta County           50       4         8     56      41   
        Washakie County        50       4         8     56      43   
        Weston County          50       4         8     56      45   

                           CENSUS2010POP  Estimates Base 2010  \
STNAME  CTYNAME                                                 
Alabama Autauga County             54571                54571   
        Baldwin County            182265               182265   
        Barbour County             27457                27457   
        Bibb County                22915                22919   
        Blount County              57322                57322   
...                                  ...                  ...   
Wyoming Sweetwater County          43806                43806   
        Teton County               21294                21294   
        Uinta County               21118                21118   
        Washakie County             8533                 8533   
        Weston County               7208                 7208   

                           POPESTIMATE2010  POPESTIMATE2011  POPESTIMATE2012  \
STNAME  CTYNAME                                                                
Alabama Autauga County               54660            55253            55175   
        Baldwin County              183193           186659           190396   
        Barbour County               27341            27226            27159   
        Bibb County                  22861            22733            22642   
        Blount County                57373            57711            57776   
...                                    ...              ...              ...   
Wyoming Sweetwater County            43593            44041            45104   
        Teton County                 21297            21482            21697   
        Uinta County                 21102            20912            20989   
        Washakie County               8545             8469             8443   
        Weston County                 7181             7114             7065   

                           ...  RDOMESTICMIG2011  RDOMESTICMIG2012  \
STNAME  CTYNAME            ...                                       
Alabama Autauga County     ...          7.242091         -2.915927   
        Baldwin County     ...         14.832960         17.647293   
        Barbour County     ...         -4.728132         -2.500690   
        Bibb County        ...         -5.527043         -5.068871   
        Blount County      ...          1.807375         -1.177622   
...                        ...               ...               ...   
Wyoming Sweetwater County  ...          1.072643         16.243199   
        Teton County       ...         -1.589565          0.972695   
        Uinta County       ...        -17.755986         -4.916350   
        Washakie County    ...        -11.637475         -0.827815   
        Weston County      ...        -11.752361         -8.040059   

                           RDOMESTICMIG2013  RDOMESTICMIG2014  \
STNAME  CTYNAME                                                 
Alabama Autauga County            -3.012349          2.265971   
        Baldwin County            21.845705         19.243287   
        Barbour County            -7.056824         -3.904

##### timeit

Nós podemos passar essas abordagens para uma função e passar essa função na função 'timeit' para contar o tempo de execução, passando também o parâmetro 'number' que diz o número de vezes que a função deve ser executada.

In [32]:
df = pd.read_csv('datasets/census.csv')

In [33]:
# Primeira abordagem

def first_approach():
    return (df.where(df['SUMLEV']==50)
             .dropna()
             .set_index(['STNAME','CTYNAME'])
             .rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'}))

timeit.timeit(first_approach, number=10)

0.2604497000002084

In [34]:
# Segunda abordagem

def second_approach():
    new_df = df[df['SUMLEV']==50]
    new_df.set_index(['STNAME','CTYNAME'], inplace=True)
    return new_df.rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'})

timeit.timeit(second_approach, number=10)

0.06734039999992092

In [35]:
# Podemos ver que a segunda abordagem, a não 'pandorable', é mais rápida


### applymap

Python tem uma poderosa função chamada 'map', que aplica uma função em cada elemento de um iterável e retorna uma lista.

Pandas tem uma função semelhante chamada **applymap**. Nessa função, passamos uma função que deverá ser operada em cada célula do DataFrame e retorna um DataFrame.

No entanto, uma função mais usada é a função **apply** que aplica uma função ao longo de um eixo do DataFrame.

In [36]:
# Vamos olhar para o DataFrame 'census'. Nesse DataFrame temos 5 colunas para a estimativa de uma população, cuja cada coluna corresponde a um ano.
import numpy as np
df=pd.read_csv('datasets/census.csv')
df.head(5)

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [37]:
# É razoável que queiramos criar novas colunas com os valores máximos e mínimos ao longo dos anos para aquela região específica.
# Primeiro precisamos criar uma função que pega uma linha, ache o valor minimo e o valor máximo e então retorna uma nova linha de dados.

def min_max(row):  
    #data é a projeção da 'row' nas colunas selecionadas   
    data = row[['POPESTIMATE2010',
                'POPESTIMATE2011',
                'POPESTIMATE2012',
                'POPESTIMATE2013',
                'POPESTIMATE2014',
                'POPESTIMATE2015']]        
    return pd.Series({'min': np.min(data), 'max': np.max(data)})
df.apply(min_max, axis=1)


,min,max
0,4785161,4858979
1,54660,55347
2,183193,203709
3,26489,27341
4,22512,22861
...,...,...
3188,43593,45162
3189,21297,23125
3190,20822,21102
3191,8316,8545


In [38]:
# Se quisermos adicionar no DataFrame original, podemos fazer desse jeito:

def min_max(row):
    data = row[['POPESTIMATE2010',
                'POPESTIMATE2011',
                'POPESTIMATE2012',
                'POPESTIMATE2013',
                'POPESTIMATE2014',
                'POPESTIMATE2015']]
    # Create a new entry for max
    row['max'] = np.max(data)
    # Create a new entry for min
    row['min'] = np.min(data)
    return row
# Now just apply the function across the dataframe
df.apply(min_max, axis='columns')

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,max,min
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594,4858979,4785161
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333,55347,54660
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499,203709,183193
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299,27341,26489
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861,22861,22512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,50,4,8,56,37,Wyoming,Sweetwater County,43806,43806,43593,...,-5.339774,-14.252889,-14.248864,1.255221,16.243199,-5.295460,-14.075283,-14.070195,45162,43593
3189,50,4,8,56,39,Wyoming,Teton County,21294,21294,21297,...,19.525929,14.143021,-0.564849,0.654527,2.408578,21.160658,16.308671,1.520747,23125,21297
3190,50,4,8,56,41,Wyoming,Uinta County,21118,21118,21102,...,-6.902954,-14.215862,-12.127022,-18.136812,-5.536861,-7.521840,-14.740608,-12.606351,21102,20822
3191,50,4,8,56,43,Wyoming,Washakie County,8533,8533,8545,...,-2.013502,-17.781491,1.682288,-11.990126,-1.182592,-2.250385,-18.020168,1.441961,8545,8316


In [39]:
# Utilizando lambda:

rows = ['POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013','POPESTIMATE2014', 
        'POPESTIMATE2015']

df.apply(lambda x: np.max(x[rows]), axis=1).head() # x é cada linha do DataFrame, para calcular o np.max precisamos projetar as colunas de interesse já que as outras colunas possuem 'strings'.

0    4858979
1      55347
2     203709
3      27341
4      22861
dtype: int64

In [40]:
# Vamos criar uma função que divida os estados em regiões:  Northeast, Midwest, South, and West.

def get_state_region(x):
    northeast = ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 
                 'Rhode Island','Vermont','New York','New Jersey','Pennsylvania']
    midwest = ['Illinois','Indiana','Michigan','Ohio','Wisconsin','Iowa',
               'Kansas','Minnesota','Missouri','Nebraska','North Dakota',
               'South Dakota']
    south = ['Delaware','Florida','Georgia','Maryland','North Carolina',
             'South Carolina','Virginia','District of Columbia','West Virginia',
             'Alabama','Kentucky','Mississippi','Tennessee','Arkansas',
             'Louisiana','Oklahoma','Texas']
    west = ['Arizona','Colorado','Idaho','Montana','Nevada','New Mexico','Utah',
            'Wyoming','Alaska','California','Hawaii','Oregon','Washington']
    
    if x in northeast:
        return "Northeast"
    elif x in midwest:
        return "Midwest"
    elif x in south:
        return "South"
    else:
        return "West"
    


In [41]:
# Agora que temos a função, vamos dizer que queremos criar uma nova coluna chamada 'Region'.
df['state_region']=df['STNAME'].apply(lambda x: get_state_region(x))

In [42]:
# Agora vamos ver os resultados
df[['STNAME','state_region']]

,STNAME,state_region
0,Alabama,South
1,Alabama,South
2,Alabama,South
3,Alabama,South
4,Alabama,South
...,...,...
3188,Wyoming,West
3189,Wyoming,West
3190,Wyoming,West
3191,Wyoming,West


# Groupby
A ideia por trás da função groupby() é que ela pega um dataframe e separa ele em pedaços baseados em alguma chave/valor.

### Spliting

In [43]:
import pandas as pd
import numpy as np

In [25]:
# Vamos olhar o DataFrame census
df=pd.read_csv('datasets/census.csv')
df=df[df['SUMLEV']==50]
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,...,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411


In [26]:
%%timeit -n 3

for state in df['STNAME'].unique():
    avg=np.average(df[df['STNAME']==state]['CENSUS2010POP'])
    print('Counties in state ' + state + 
          ' have an average population of ' + str(avg))

NameError: name 'np' is not defined

In [27]:
# Agora vamos tentar pelo método groupby()

#Precisamos dizer para o Pandas que estamos interessados em agrupar pelo nome do estado, isso é o 'split'
for result in df.groupby('STNAME'):
    firstState=result
    break
print(type(firstState))
print(firstState[0])
firstState[1]

<class 'tuple'>
Alabama


,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,...,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,50,3,6,1,125,Alabama,Tuscaloosa County,194656,194653,194977,...,3.983504,4.256278,5.261075,4.787490,1.884402,5.306232,5.202679,6.333332,6.057539,3.158710
64,50,3,6,1,127,Alabama,Walker County,67023,67023,67004,...,-4.579296,-4.336533,-0.666096,-3.300481,-3.087245,-4.085450,-3.869753,-0.181663,-2.692097,-2.460626
65,50,3,6,1,129,Alabama,Washington County,17581,17583,17610,...,-14.708407,-13.062030,-11.181733,1.067236,-0.950486,-13.849940,-12.075121,-10.122411,2.193763,0.118811
66,50,3,6,1,131,Alabama,Wilcox County,11670,11665,11557,...,-7.029724,-10.239356,-12.314507,-13.835235,0.090453,-7.029724,-10.239356,-12.314507,-13.835235,0.090453


In [47]:
# Podemos ver que a função groupby cria um objeto com tuplas, e em cada tupla temos o nome do estado na primeira posição e o 
# DataFrame contendo as informações do estado na segunda posição.


In [29]:
import numpy as np

In [30]:
%%timeit
# Voltando ao cálculo da média
for group,frame in df.groupby("STNAME"):
    avg=np.average(frame['CENSUS2010POP'])
    print('Counties in state ' + group + 
          ' have an average population of ' + str(avg))

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

...

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

### Using functions to split our database

In [49]:
# Podemos passar uma função como parâmetro da função groupby() para segmentar os nossos dados.

# Suponhamos que queremos trabalhar em apenas 1/3 o DataFrame. Podemos criar uma função que retorna um número entre 0 e 2
# baseado no primeiro caractere do nome do estado.
# Então podemos dizer ao groupby para usar essa função para segmentar os nossos dados.
# Importante: Precisamos setar o indice como a coluna que iremos agrupar
df=df.reset_index()
df=df.set_index('STNAME')

# Essa função retorna uma variável que será o nome do grupo, nesse caso 0,1,2.
# O groupby irá armazenar as linhas do DataFrame nesses grupos de acordo com a função.
def set_batch_number(item):
    if item[0]<'M':
        return 0
    if item[0]<'Q':
        return 1
    return 2

for group,frame in df.groupby(set_batch_number):
    print('There are ' + str(len(frame)) + ' records in group ' + str(group) + ' for processing.')
    print(group)
    print(frame)

There are 1177 records in group 0 for processing.
0
           index  SUMLEV  REGION  DIVISION  STATE  COUNTY  \
STNAME                                                      
Alabama        1      50       3         6      1       1   
Alabama        2      50       3         6      1       3   
Alabama        3      50       3         6      1       5   
Alabama        4      50       3         6      1       7   
Alabama        5      50       3         6      1       9   
...          ...     ...     ...       ...    ...     ...   
Louisiana   1191      50       3         7     22     119   
Louisiana   1192      50       3         7     22     121   
Louisiana   1193      50       3         7     22     123   
Louisiana   1194      50       3         7     22     125   
Louisiana   1195      50       3         7     22     127   

                           CTYNAME  CENSUS2010POP  ESTIMATESBASE2010  \
STNAME                                                                 
Alabama   

In [50]:
# Nesse exemplo, temos os estados com o primeiro caractere de A até L incluídos no grupo 1, de M até P incluídos no grupo 2 e 
# o resto no grupo 3.

# Se nenhuma coluna for passada, groupby irá usar a coluna index automaticamente.

### Another example

In [32]:
# Nesse exemplo, queremos usar o dataset de casas do airbnb.
# Aqui temos duas colunas de interesse, a 'cancellation_policy' e a 'review_scores_value'.
import pandas as pd
df=pd.read_csv('datasets/listings.csv')
df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,9.0,f,NaN,NaN,t,moderate,f,f,1,1.30
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,10.0,f,NaN,NaN,f,moderate,t,f,1,0.47
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,...,10.0,f,NaN,NaN,f,moderate,f,f,1,1.00
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",...,10.0,f,NaN,NaN,f,flexible,f,f,1,2.25


In [33]:
# Então, como podemos agrupar com essas duas colunas?
# Podemos fazer de dois jeitos

# A primeira abordagem seria promover ambas como multi-index e chamar o groupby() utilizando o parâmetro 'level' passando (0,1).
# A segunda abordagem seria passar como parâmetro do groupby as colunas dentro de uma lista: df.groupby(["cancellation_policy","review_scores_value"]):


In [34]:
# Vamos fazer do primeiro jeito  
df=df.reset_index()
df=df.set_index(["cancellation_policy","review_scores_value"])
for result in df.groupby(level=(0,1)):
    print(result[0])
    break
    

('flexible', 2.0)


In [35]:
df

index        id  \
cancellation_policy review_scores_value                    
moderate            NaN                      0  12147973   
                    9.0                      1   3075044   
                    10.0                     2      6976   
                    10.0                     3   1436513   
flexible            10.0                     4   7651065   
...                                        ...       ...   
strict              9.0                   3580   8373729   
                    NaN                   3581  14844274   
flexible            NaN                   3582  14585486   
strict              7.0                   3583  14603878   
flexible            NaN                   3584  14504422   

                                                                   listing_url  \
cancellation_policy review_scores_value                                          
moderate            NaN                  https://www.airbnb.com/rooms/12147973   
                    9.0                   https://www.airbnb.com/rooms/3075044   
                    10.0                     https://www.airbnb.com/rooms/6976   
                    10.0                  https://www.airbnb.com/rooms/1436513   
flexible            10.0                  https://www.airbnb.com/rooms/7651065   
...                                                                        ...   
strict              9.0                   https://www.airbnb.com/rooms/8373729   
                    NaN                  https://www.airbnb.com/rooms/14844274   
flexible            NaN                  https://www.airbnb.com/rooms/14585486   
strict              7.0                  https://www.airbnb.com/rooms/14603878   
flexible            NaN                  https://www.airbnb.com/rooms/14504422   

                                              scrape_id last_scraped  \
cancellation_policy review_scores_value                                
moderate            NaN                  20160906204935   2016-09-07   
                    9.0                  20160906204935   2016-09-07   
                    10.0                 20160906204935   2016-09-07   
                    10.0                 20160906204935   2016-09-07   
flexible            10.0                 20160906204935   2016-09-07   
...                                                 ...          ...   
strict              9.0                  20160906204935   2016-09-07   
                    NaN                  20160906204935   2016-09-07   
flexible            NaN                  20160906204935   2016-09-07   
strict              7.0                  20160906204935   2016-09-07   
flexible            NaN                  20160906204935   2016-09-07   

                                                                                  name  \
cancellation_policy review_scores_value                                                  
moderate            NaN                                     Sunny Bungalow in the City   
                    9.0                              Charming room in pet friendly apt   
                    10.0                              Mexican Folk Art Haven in Boston   
                    10.0                 Spacious Sunny Bedroom Suite in Historic Home   
flexible            10.0                                           Come Home to Boston   
...                                                                                ...   
strict              9.0                                           Big cozy room near T   
                    NaN                            BU Apartment DexterPark Bright room   
flexible            NaN                                       Gorgeous funky apartment   
strict              7.0                          Great Location; Train and Restaurants   
flexible            NaN                             (K1) Private Room near Harvard/MIT   

                                                                                   su

In [36]:
# No caso multi-index, o primeiro elemento da tupla é uma tupla contendo os nomes dos indices

In [37]:
# Voltando ao exemplo:
for group, frame in df.groupby(level=(0,1)):
    print(group)

('flexible', 2.0)
('flexible', 4.0)
('flexible', 5.0)
('flexible', 6.0)
('flexible', 7.0)
('flexible', 8.0)
('flexible', 9.0)
('flexible', 10.0)
('moderate', 2.0)
('moderate', 4.0)
('moderate', 6.0)
('moderate', 7.0)
('moderate', 8.0)
('moderate', 9.0)
('moderate', 10.0)
('strict', 2.0)
('strict', 3.0)
('strict', 4.0)
('strict', 5.0)
('strict', 6.0)
('strict', 7.0)
('strict', 8.0)
('strict', 9.0)
('strict', 10.0)
('super_strict_30', 6.0)
('super_strict_30', 7.0)
('super_strict_30', 8.0)
('super_strict_30', 9.0)
('super_strict_30', 10.0)


In [38]:
# E se quisermos agrupar nas colunas 'cancelation policy' e 'review scores', mas separar a coluna 'review scores' em 2 casos: igual e diferente de 10.
# Nesse caso fazemos o uso de funções novamente

def grouping_fun(item):
    # Check the "review_scores_value" portion of the index. item is in the format of
    # (cancellation_policy,review_scores_value)
    print(f'k {item}')
    if item[1] == 10.0:
        return (item[0],"10.0")
    else:
        return (item[0],"not 10.0")
    
for group, frame in df.groupby(grouping_fun):
    print(group)

k ('moderate', nan)
k ('moderate', 9.0)
k ('moderate', 10.0)
k ('moderate', 10.0)
k ('flexible', 10.0)
k ('flexible', 10.0)
k ('strict', 9.0)
k ('moderate', 10.0)
k ('moderate', 10.0)
k ('strict', 9.0)
k ('flexible', 10.0)
k ('flexible', 9.0)
k ('flexible', 10.0)
k ('moderate', 10.0)
k ('moderate', 9.0)
k ('strict', 10.0)
k ('flexible', 10.0)
k ('flexible', 10.0)
k ('strict', nan)
k ('strict', 10.0)
k ('flexible', 10.0)
k ('moderate', 10.0)
k ('strict', 9.0)
k ('flexible', 10.0)
k ('moderate', 10.0)
k ('moderate', 10.0)
k ('strict', 9.0)
k ('moderate', 10.0)
k ('flexible', 9.0)
k ('flexible', 10.0)
k ('flexible', 9.0)
k ('strict', 10.0)
k ('flexible', nan)
k ('strict', 10.0)
k ('flexible', 10.0)
k ('strict', 9.0)
k ('strict', nan)
k ('strict', 9.0)
k ('strict', 9.0)
k ('moderate', 10.0)
k ('flexible', 9.0)
k ('flexible', 10.0)
k ('flexible', 10.0)
k ('flexible', 9.0)
k ('strict', 9.0)
k ('flexible', 9.0)
k ('moderate', 10.0)
k ('strict', 10.0)
k ('flexible', 10.0)
k ('flexible', 10.0)


# Applying
Existem 3 categorias de processamento de dados: 'aggregation', 'transformation' e 'filtration'.


### Aggregation

In [39]:
# Utilizamos o método agg() no objeto groupby().
# Podemos passar como parâmetro do agg() um dicionário com as colunas que queremos e a função que desejamos aplicar a elas.

df=df.reset_index()

#Vamos agrupar por 'cancellation_policy' e achar a média da coluna 'review_scores'.
df.groupby("cancellation_policy").agg({"review_scores_value":np.average})

,review_scores_value
cancellation_policy,
flexible,NaN
moderate,NaN
strict,NaN
super_strict_30,NaN


In [58]:
# Isso aconteceu porque 'np.average' não ignora os NaN, precisamos usar a função 'np.nanmean' ou 'np.mean'
df.groupby("cancellation_policy").agg({"review_scores_value":np.nanmean})

,review_scores_value
cancellation_policy,
flexible,9.237421
moderate,9.307398
strict,9.081441
super_strict_30,8.537313


In [59]:
# Podemos extender o dicionário passando múltiplas funções ou colunas

df.groupby("cancellation_policy").agg({"review_scores_value":(np.nanmean,np.nanstd),
                                      "reviews_per_month":np.nanmean})

review_scores_value           reviews_per_month
                                nanmean    nanstd           nanmean
cancellation_policy                                                
flexible                       9.237421  1.096271          1.829210
moderate                       9.307398  0.859859          2.391922
strict                         9.081441  1.040531          1.873467
super_strict_30                8.537313  0.840785          0.340143

### Transformation

In [44]:
# Enquanto agg() retorna um único valor por coluna,ou uma única linha por DataFrame/grupo.
# transform() retorna um objeto do mesmo tamanho do grupo, isso torna a combinação de dados mais fácil.

df=pd.read_csv('datasets/listings.csv')
# Exemplo: Vamos supor que queiramos incluir a média dos valores, mas preservar o DataFrame
cols=['cancellation_policy','review_scores_value']
df[cols].groupby('cancellation_policy').transform(np.mean)

,review_scores_value
0,9.307398
1,9.307398
2,9.307398
3,9.307398
4,9.237421
...,...
3580,9.081441
3581,9.081441
3582,9.237421
3583,9.081441


In [49]:
# Aqui, os indices são iguais ao DataFrame original, então podemos junta-los.
transform_df=df[cols].groupby('cancellation_policy').transform(np.mean)
transform_df.rename({'review_scores_value':'mean_review_scores'}, axis='columns', inplace=True)

df=df.merge(transform_df, left_index=True, right_index=True)
df[['cancellation_policy','mean_review_scores']]
df

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,mean_review_scores
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,f,NaN,NaN,f,moderate,f,f,1,NaN,9.307398
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,f,NaN,NaN,t,moderate,f,f,1,1.30,9.307398
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,f,NaN,NaN,f,moderate,t,f,1,0.47,9.307398
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,...,f,NaN,NaN,f,moderate,f,f,1,1.00,9.307398
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",...,f,NaN,NaN,f,flexible,f,f,1,2.25,9.237421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3580,8373729,https://www.airbnb.com/rooms/8373729,20160906204935,2016-09-07,Big cozy room near T,5 min walking to Orange Line subway with 2 sto...,NaN,5 min walking to Orange Line subway with 2 sto...,none,NaN,...,f,NaN,NaN,t,strict,f,f,8,0.34,9.081441
3581,14844274,https://www.airbnb.com/rooms/14844274,20160906204935,2016-09-07,BU Apartment DexterPark Bright room,"Most popular apartment in BU, best located in ...",Best location in BU,"Most popular apartment in BU, best located in ...",none,NaN,...,f,NaN,NaN,f,strict,f,f,2,NaN,9.081441
3582,14585486,https://www.airbnb.com/rooms/14585486,20160906204935,2016-09-07,Gorgeous funky apartment,Funky little apartment close to public transpo...,Modern and relaxed space with many facilities ...,Funky little apartment close to public transpo...,none,"Cambridge is a short walk into Boston, and set...",...,f,NaN,NaN,f,flexible,f,f,1,NaN,9.237421
3583,14603878,https://www.airbnb.com/rooms/14603878,20160906204935,2016-09-07,Great Location; Train and Restaurants,"My place is close to Taco Loco Mexican Grill, ...",NaN,"My place is close to Taco Loco Mexican Grill, ...",none,NaN,...,f,NaN,NaN,f,strict,f,f,1,2.00,9.081441


### Filtering
A função filter() recebe uma função que é aplicada ao DataFrame e retorna True ou False

In [50]:
df

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,mean_review_scores
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,f,NaN,NaN,f,moderate,f,f,1,NaN,9.307398
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,f,NaN,NaN,t,moderate,f,f,1,1.30,9.307398
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,f,NaN,NaN,f,moderate,t,f,1,0.47,9.307398
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,...,f,NaN,NaN,f,moderate,f,f,1,1.00,9.307398
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",...,f,NaN,NaN,f,flexible,f,f,1,2.25,9.237421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3580,8373729,https://www.airbnb.com/rooms/8373729,20160906204935,2016-09-07,Big cozy room near T,5 min walking to Orange Line subway with 2 sto...,NaN,5 min walking to Orange Line subway with 2 sto...,none,NaN,...,f,NaN,NaN,t,strict,f,f,8,0.34,9.081441
3581,14844274,https://www.airbnb.com/rooms/14844274,20160906204935,2016-09-07,BU Apartment DexterPark Bright room,"Most popular apartment in BU, best located in ...",Best location in BU,"Most popular apartment in BU, best located in ...",none,NaN,...,f,NaN,NaN,f,strict,f,f,2,NaN,9.081441
3582,14585486,https://www.airbnb.com/rooms/14585486,20160906204935,2016-09-07,Gorgeous funky apartment,Funky little apartment close to public transpo...,Modern and relaxed space with many facilities ...,Funky little apartment close to public transpo...,none,"Cambridge is a short walk into Boston, and set...",...,f,NaN,NaN,f,flexible,f,f,1,NaN,9.237421
3583,14603878,https://www.airbnb.com/rooms/14603878,20160906204935,2016-09-07,Great Location; Train and Restaurants,"My place is close to Taco Loco Mexican Grill, ...",NaN,"My place is close to Taco Loco Mexican Grill, ...",none,NaN,...,f,NaN,NaN,f,strict,f,f,1,2.00,9.081441


In [62]:
# Por exemplo, se quisermos a média acima de 9 nos nossos resultados
filtering_df=df.groupby('cancellation_policy').filter(lambda x: np.nanmean(x['review_scores_value'])>9.2)
filtering_df['cancellation_policy'].unique()


array(['moderate', 'flexible'], dtype=object)

### Applying
A função apply() permite aplicar uma função arbitrária em cada grupo, e retornar o resultado em um único DataFrame onde o indice é preservado.

In [63]:
import pandas as pd
df=pd.read_csv("datasets/listings.csv")
df=df[['cancellation_policy','review_scores_value']]
df.head()

,cancellation_policy,review_scores_value
0,moderate,NaN
1,moderate,9.0
2,moderate,10.0
3,moderate,10.0
4,flexible,10.0


In [64]:
# Vamos calcular a média e o desvio padrão

def calc_mean_review_scores(group):
    #group é o DataFrame   
    avg=np.nanmean(group["review_scores_value"])   
    group['mean_diff']=np.abs(avg-group["review_scores_value"])
    return group
df.groupby('cancellation_policy').apply(calc_mean_review_scores).head()

C:\Users\T-Gamer\AppData\Local\Temp\ipykernel_11280\2569121930.py:8: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df.groupby('cancellation_policy').apply(calc_mean_review_scores).head()


,cancellation_policy,review_scores_value,mean_diff
0,moderate,NaN,NaN
1,moderate,9.0,0.307398
2,moderate,10.0,0.692602
3,moderate,10.0,0.692602
4,flexible,10.0,0.762579


# Scales
Pandas suporta diferentes tipos de dados como string,inteiros,floats.
O que não é capturado é o que chamamos de **escala dos dados**.

Exemplo: Notas A+, A, A-, B+, B, B- de um estudante não tem a mesma escala. A diferença entre as notas A e A- e a diferença entre as notas A- e B+ não são iguais.

Existem pelo menos 4 tipos diferentes de escala:


**Ratio scale**

-As unidades de medida são igualmente espaçadas e operações matemáticas são válidas.

Exemplo: Peso e Altura

**Interval Scale**

-As unidades de medida são igualmente espaçadas, mas não existe uma clara ausência de valor e as operações matemáticas não são válidas.

Exemplo: Temperatura -> Não existe uma ausência de temperatura, 0° representa uma temperatura. Outro exemplo é a bússola, não existe ausência de direção.


**Ordinal Scale**

-A ordem dos valores é importante, mas a diferença entre os valores não é igualmente espaçada

Exemplo: Notas de um aluno (A+,A,A-)

**Nominal Scale**

-Também chamada de **categorical data**.

-Exemplo : Times de futebol; tem um número limitado de times de futebol, mas mudar a ordem deles ou aplicar operações matemáticas não tem sentido

In [51]:
import pandas as pd

df=pd.DataFrame(['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D'],
                index=['excellent', 'excellent', 'excellent', 'good', 'good', 'good', 
                       'ok', 'ok', 'ok', 'poor', 'poor'],
               columns=["Grades"])
df

,Grades
excellent,A+
excellent,A
excellent,A-
good,B+
good,B
good,B-
ok,C+
ok,C
ok,C-
poor,D+


In [52]:
# Podemos dizer ao Pandas que queremos mudar o 'type' para 'category' usando a função astype()
df["Grades"].astype("category").head()

excellent    A+
excellent     A
excellent    A-
good         B+
good          B
Name: Grades, dtype: category
Categories (11, object): ['A', 'A+', 'A-', 'B', ..., 'C+', 'C-', 'D', 'D+']

In [67]:
# Podemos ver que o 'dtype' mudou para 'category' e temos 11 categorias.

### Boolean Mask

In [53]:
# O que acontece se quisermos pegar as notas desse aluno que foram maiores do que C?
df[df["Grades"]>"C"]

,Grades
ok,C+
ok,C-
poor,D+
poor,D


In [69]:
# Errado, o Pandas não reconhece a ordem dos dados

In [54]:
# Podemos dizer ao Pandas também, que nossos dados são ordenados criando um parâmetro com a ordem e 'ordered=True'.

my_categories=pd.CategoricalDtype(categories=['D', 'D+', 'C-', 'C', 'C+', 'B-', 'B', 'B+', 'A-', 'A', 'A+'],ordered=True)

# E então, podemos passar como parâmetro para a função astype().
grades=df['Grades'].astype(my_categories)
grades.head

<bound method NDFrame.head of excellent    A+
excellent     A
excellent    A-
good         B+
good          B
good         B-
ok           C+
ok            C
ok           C-
poor         D+
poor          D
Name: Grades, dtype: category
Categories (11, object): ['D' < 'D+' < 'C-' < 'C' ... 'B+' < 'A-' < 'A' < 'A+']>

In [56]:
df[df["Grades"]>"C"]

,Grades
ok,C+
ok,C-
poor,D+
poor,D


In [71]:
# Agora, Pandas não está se preocupando somente com as 11 categorias, mas também com a ordem delas.

In [57]:
# Então, agora sim podemos fazer a consulta das notas maiores do que 'C'.
grades[grades>"C"]

excellent    A+
excellent     A
excellent    A-
good         B+
good          B
good         B-
ok           C+
Name: Grades, dtype: category
Categories (11, object): ['D' < 'D+' < 'C-' < 'C' ... 'B+' < 'A-' < 'A' < 'A+']

### Another scale-based operation

Converte a escala de algo que está na escala de intervalo ou razão,como uma nota numérica, em uma escala que é 'categorical'.
Exemplo: Se estivermos visualizando frequências de categorias.

Pandas tem uma função chamada cut, que recebe como argumento alguma estrutura tipo array como uma coluna de um dataframe ou um
Serie

In [58]:
# Vamos pegar o dataset Census.
# Vimos que poderíamos agrupar por estado e depois agregar para obter um lista do tamanho médio do condado por estado.

import numpy as np
df=pd.read_csv("datasets/census.csv")

df=df[df['SUMLEV']==50]

df=df.groupby('STNAME')['CENSUS2010POP'].agg(np.average)
df.head()

STNAME
Alabama        71339.343284
Alaska         24490.724138
Arizona       426134.466667
Arkansas       38878.906667
California    642309.586207
Name: CENSUS2010POP, dtype: float64

In [74]:
# Agora, podemos usar o cut, colocando bins=10
pd.cut(df,10)

STNAME
Alabama                   (11706.087, 75333.413]
Alaska                    (11706.087, 75333.413]
Arizona                 (390320.176, 453317.529]
Arkansas                  (11706.087, 75333.413]
California              (579312.234, 642309.586]
Colorado                 (75333.413, 138330.766]
Connecticut             (390320.176, 453317.529]
Delaware                (264325.471, 327322.823]
District of Columbia    (579312.234, 642309.586]
Florida                 (264325.471, 327322.823]
Georgia                   (11706.087, 75333.413]
Hawaii                  (264325.471, 327322.823]
Idaho                     (11706.087, 75333.413]
Illinois                 (75333.413, 138330.766]
Indiana                   (11706.087, 75333.413]
Iowa                      (11706.087, 75333.413]
Kansas                    (11706.087, 75333.413]
Kentucky                  (11706.087, 75333.413]
Louisiana                 (11706.087, 75333.413]
Maine                    (75333.413, 138330.766]
Maryland     

In [75]:
# Aqui vemos que estados como o alabama e o alasca se enquadram na mesma categoria, enquanto a Califórnia e o distrito de columbia
# se enquadram em uma categoria muito diferente. 
# Agora, cortar é apenas uma maneira de criar categorias a partir de seus dados, e existem muitos outros métodos. 
# Por exemplo, cut fornece dados de intervalo, onde o espaçamento entre cada categoria é igual. 
# Mas às vezes você quer formar categorias com base na frequência – você quer que o número de itens em cada caixa seja o mesmo,
# em vez do espaçamento entre as caixas. Isso realmente depende da forma dos seus dados e do que você planeja fazer com eles.

# Pivot Table
Um **pivot table** é um jeito de resumir os dados para um propósito particular. Se faz necessário o uso da função agg().
Um **pivot table** é um DataFrame, onde as linhas representam uma variável que estamos interessados,as colunas outra variável e as células são valores agregados.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Vamos trabalhar com o dataset "Times Higher Education World University Ranking", que mostra o ranking das universidades
df = pd.read_csv('datasets/cwurData.csv')
df.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


In [3]:
# Vamos supor que queremos criar uma nova coluna chamada "Rank_level" ondas instituições com "world_rank" entre 1 e 100 são 
# categorizadas como 'first tier', as com "world_rank" entre 101 e 200 serão categorizadas como 'second tier' e assim por diante.

# Uma das soluções que vimos é:
def create_category(ranking):    
    if (ranking >= 1) & (ranking <= 100):
        return "First Tier Top University"
    elif (ranking >= 101) & (ranking <= 200):
        return "Second Tier Top University"
    elif (ranking >= 201) & (ranking <= 300):
        return "Third Tier Top University"
    return "Other Top Unversity"

df['Rank_Level'] = df['world_rank'].apply(lambda x: create_category(x))

df[['Rank_Level','score','country']]

,Rank_Level,score,country
0,First Tier Top University,100.00,USA
1,First Tier Top University,91.67,USA
2,First Tier Top University,89.50,USA
3,First Tier Top University,86.17,United Kingdom
4,First Tier Top University,85.21,USA
...,...,...,...
2195,Other Top Unversity,44.03,Portugal
2196,Other Top Unversity,44.03,Egypt
2197,Other Top Unversity,44.03,Brazil
2198,Other Top Unversity,44.02,Spain


In [4]:
# Um pivot table nos permite dinamizar uma dessas colunas em um novo cabeçalho de coluna e compará-la com 
# outra coluna como índices de linha

#Exemplo
# Vamos comparar o 'rank_level' com os países em termos do 'overall score'
#Ou seja, nesse caso queremos que nosso indice sejam os países, o cabeçalho seja o rank level e os valores das células a média do 'overall score'.

df.pivot_table(values='score', index='country', columns='Rank_Level', aggfunc=[np.mean]).head()

mean                      \
Rank_Level First Tier Top University Other Top Unversity   
country                                                    
Argentina                        NaN           44.672857   
Australia                    47.9425           44.645750   
Austria                          NaN           44.864286   
Belgium                      51.8750           45.081000   
Brazil                           NaN           44.499706   

                                                                 
Rank_Level Second Tier Top University Third Tier Top University  
country                                                          
Argentina                         NaN                       NaN  
Australia                     49.2425                 47.285000  
Austria                           NaN                 47.066667  
Belgium                       49.0840                 46.746667  
Brazil                        49.5650                       NaN

In [80]:
#IMPORTANTE: o parâmetro 'aggfunc' é igual a 'mean' por default,então nesse caso não precisaríamos passar o 'np.mean'

In [81]:
# Podemos ver um DataFrame hierárquico onde os indices(ou linhas) são por países, e as linhas possuem 2 levels: o level mais acima
# indicando 'mean' e o segundo os 'ranks'

In [82]:
# Podemos aplicar mais de uma função no parâmetro 'aggfunc'.
# Vamos tentar

df.pivot_table(values='score', index='country', columns='Rank_Level', aggfunc=[np.mean, np.max]).head()

mean                      \
Rank_Level First Tier Top University Other Top Unversity   
country                                                    
Argentina                        NaN           44.672857   
Australia                    47.9425           44.645750   
Austria                          NaN           44.864286   
Belgium                      51.8750           45.081000   
Brazil                           NaN           44.499706   

                                                                 \
Rank_Level Second Tier Top University Third Tier Top University   
country                                                           
Argentina                         NaN                       NaN   
Australia                     49.2425                 47.285000   
Austria                           NaN                 47.066667   
Belgium                       49.0840                 46.746667   
Brazil                        49.5650                       NaN   

                                amax                      \
Rank_Level First Tier Top University Other Top Unversity   
country                                                    
Argentina                        NaN               45.66   
Australia                      51.61               45.97   
Austria                          NaN               46.29   
Belgium                        52.03               46.21   
Brazil                           NaN               46.08   

                                                                 
Rank_Level Second Tier Top University Third Tier Top University  
country                                                          
Argentina                         NaN                       NaN  
Australia                       50.40                     47.47  
Austria                           NaN                     47.78  
Belgium                         49.73                     47.14  
Brazil                          49.82                       NaN

In [83]:
# Agora temos a média e o valor máximo. 
# Se quisermos ver uma média das médias, e o máximo dos valores máximos, podemos indicar utilizando o parâmetro margins=True.
df.pivot_table(values='score', index='country', columns='Rank_Level', aggfunc=[np.mean, np.max], 
               margins=True).head()

mean                      \
Rank_Level First Tier Top University Other Top Unversity   
country                                                    
Argentina                        NaN           44.672857   
Australia                    47.9425           44.645750   
Austria                          NaN           44.864286   
Belgium                      51.8750           45.081000   
Brazil                           NaN           44.499706   

                                                                            \
Rank_Level Second Tier Top University Third Tier Top University        All   
country                                                                      
Argentina                         NaN                       NaN  44.672857   
Australia                     49.2425                 47.285000  45.825517   
Austria                           NaN                 47.066667  45.139583   
Belgium                       49.0840                 46.746667  47.011000   
Brazil                        49.5650                       NaN  44.781111   

                                amax                      \
Rank_Level First Tier Top University Other Top Unversity   
country                                                    
Argentina                        NaN               45.66   
Australia                      51.61               45.97   
Austria                          NaN               46.29   
Belgium                        52.03               46.21   
Brazil                           NaN               46.08   

                                                                        
Rank_Level Second Tier Top University Third Tier Top University    All  
country                                                                 
Argentina                         NaN                       NaN  45.66  
Australia                       50.40                     47.47  51.61  
Austria                           NaN                     47.78  47.78  
Belgium                         49.73                     47.14  52.03  
Brazil                          49.82                       NaN  49.82

In [5]:
# Pivot table é só um DataFrame multi-level, então podemos acessar as células de uma maneira semelhante a que faziamos antes.
new_df=df.pivot_table(values='score', index='country', columns='Rank_Level', aggfunc=[np.mean, np.max], 
               margins=True)
new_df.head()

mean                      \
Rank_Level First Tier Top University Other Top Unversity   
country                                                    
Argentina                        NaN           44.672857   
Australia                    47.9425           44.645750   
Austria                          NaN           44.864286   
Belgium                      51.8750           45.081000   
Brazil                           NaN           44.499706   

                                                                            \
Rank_Level Second Tier Top University Third Tier Top University        All   
country                                                                      
Argentina                         NaN                       NaN  44.672857   
Australia                     49.2425                 47.285000  45.825517   
Austria                           NaN                 47.066667  45.139583   
Belgium                       49.0840                 46.746667  47.011000   
Brazil                        49.5650                       NaN  44.781111   

                                amax                      \
Rank_Level First Tier Top University Other Top Unversity   
country                                                    
Argentina                        NaN               45.66   
Australia                      51.61               45.97   
Austria                          NaN               46.29   
Belgium                        52.03               46.21   
Brazil                           NaN               46.08   

                                                                        
Rank_Level Second Tier Top University Third Tier Top University    All  
country                                                                 
Argentina                         NaN                       NaN  45.66  
Australia                       50.40                     47.47  51.61  
Austria                           NaN                     47.78  47.78  
Belgium                         49.73                     47.14  52.03  
Brazil                          49.82                       NaN  49.82

In [11]:
new_df.unstack().unstack()

country                          Argentina  Australia    Austria    Belgium  \
     Rank_Level                                                               
mean First Tier Top University         NaN  47.942500        NaN  51.875000   
     Other Top Unversity         44.672857  44.645750  44.864286  45.081000   
     Second Tier Top University        NaN  49.242500        NaN  49.084000   
     Third Tier Top University         NaN  47.285000  47.066667  46.746667   
     All                         44.672857  45.825517  45.139583  47.011000   
amax First Tier Top University         NaN  51.610000        NaN  52.030000   
     Other Top Unversity         45.660000  45.970000  46.290000  46.210000   
     Second Tier Top University        NaN  50.400000        NaN  49.730000   
     Third Tier Top University         NaN  47.470000  47.780000  47.140000   
     All                         45.660000  51.610000  47.780000  52.030000   

country                             Brazil  Bulgaria     Canada    Chile  \
     Rank_Level                                                            
mean First Tier Top University         NaN       NaN  53.633846      NaN   
     Other Top Unversity         44.499706    44.335  44.760541  44.7675   
     Second Tier Top University  49.565000       NaN  49.218182      NaN   
     Third Tier Top University         NaN       NaN  46.826364      NaN   
     All                         44.781111    44.335  47.359306  44.7675   
amax First Tier Top University         NaN       NaN  60.870000      NaN   
     Other Top Unversity         46.080000    44.480  45.740000  45.3300   
     Second Tier Top University  49.820000       NaN  51.230000      NaN   
     Third Tier Top University         NaN       NaN  47.690000      NaN   
     All                         49.820000    44.480  60.870000  45.3300   

country                              China  Colombia  ...  Switzerland  \
     Rank_Level                                       ...                
mean First Tier Top University   53.592500       NaN  ...    54.005000   
     Other Top Unversity         44.564267   44.4325  ...    44.625000   
     Second Tier Top University  47.868000       NaN  ...    48.184000   
     Third Tier Top University   46.926250       NaN  ...    47.930000   
     All                         44.992575   44.4325  ...    51.208846   
amax First Tier Top University   55.300000       NaN  ...    72.180000   
     Other Top Unversity         45.920000   44.8500  ...    45.050000   
     Second Tier Top University  48.140000       NaN  ...    49.170000   
     Third Tier Top University   47.760000       NaN  ...    47.930000   
     All                         55.300000   44.8500  ...    72.180000   

country                             Taiwan   Thailand  Turkey         USA  \
     Rank_Level                                                             
mean First Tier Top University   54.210000        NaN     NaN   61.066726   
     Other Top Unversity         44.476667  44.830000  44.481   44.871718   
     Second Tier Top University        NaN        NaN     NaN   49.069524   
     Third Tier Top University   47.065000  46.550000     NaN   46.818333   
     All                         45.012391  45.116667  44.481   51.839860   
amax First Tier Top University   54.230000        NaN     NaN  100.000000   
     Other Top Unversity         45.490000  45.860000  45.570   46.300000   
     Second Tier Top University        NaN        NaN     NaN   51.270000   
     Third Tier Top University   47.160000  46.550000     NaN   47.820000   
     All                         54.230000  46.550000  45.570  100.000000   

country                          Uganda  United Arab Emirates  United Kingdom  \
     Rank_Level                                                                 
mean First Tier Top University      NaN                   NaN       63.937931   
     Other Top Unversity          44.28                 44.22       44.881299   


In [85]:
# Vamos olhar os indices
print(new_df.index)

# Vamos olhar as colunas
print(new_df.columns)

Index(['Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil', 'Bulgaria',
       'Canada', 'Chile', 'China', 'Colombia', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Egypt', 'Estonia', 'Finland', 'France',
       'Germany', 'Greece', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Iran',
       'Ireland', 'Israel', 'Italy', 'Japan', 'Lebanon', 'Lithuania',
       'Malaysia', 'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Poland',
       'Portugal', 'Puerto Rico', 'Romania', 'Russia', 'Saudi Arabia',
       'Serbia', 'Singapore', 'Slovak Republic', 'Slovenia', 'South Africa',
       'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Taiwan', 'Thailand',
       'Turkey', 'USA', 'Uganda', 'United Arab Emirates', 'United Kingdom',
       'Uruguay', 'All'],
      dtype='object', name='country')
MultiIndex([('mean',  'First Tier Top University'),
            ('mean',        'Other Top Unversity'),
            ('mean', 'Second Tier Top University'),
            ('mean',  'Thir

In [86]:
# Podemos ver que as colunas são hierárquicas.
# A coluna mais alta possui duas categorias: mean e max; a coluna mais baixa possui 4 categorias que são os 4 'rank levels'.

# Como fazer para acessar as células?
# Exemplo: Como acessar a coluna da média do rank 'First Tier Top University'
# Precisamos fazer duas projeções, a primeira na 'mean' e a segunda na coluna 'First Tier Top University'
new_df['mean']['First Tier Top University'].head()

country
Argentina        NaN
Australia    47.9425
Austria          NaN
Belgium      51.8750
Brazil           NaN
Name: First Tier Top University, dtype: float64

In [87]:
# E se quisermos achar o país (indice) que possui a maior média de 'overallscore' na coluna 'First Tier Top University'?
# Podemos usar a função idxmax()

np.max(new_df['mean']['First Tier Top University']) # nos retorna a média

new_df['mean']['First Tier Top University'].idxmax() # nos retorna o indica cuja a média é máxima

'United Kingdom'

### Stack and Unstack

In [88]:
# Se quisermos mudar o 'shape' do 'pivot table', podemos usar as funções stack e unstack.
# "Stacking" gira o cabeçalho mais baixa para se tornar o índice mais interno.
# "Unstacking" gira o índice mais interno para se tornar o cabeçalho mais baixo

In [98]:
new_df.head()

mean                      \
Rank_Level First Tier Top University Other Top Unversity   
country                                                    
Argentina                        NaN           44.672857   
Australia                    47.9425           44.645750   
Austria                          NaN           44.864286   
Belgium                      51.8750           45.081000   
Brazil                           NaN           44.499706   

                                                                            \
Rank_Level Second Tier Top University Third Tier Top University        All   
country                                                                      
Argentina                         NaN                       NaN  44.672857   
Australia                     49.2425                 47.285000  45.825517   
Austria                           NaN                 47.066667  45.139583   
Belgium                       49.0840                 46.746667  47.011000   
Brazil                        49.5650                       NaN  44.781111   

                                amax                      \
Rank_Level First Tier Top University Other Top Unversity   
country                                                    
Argentina                        NaN               45.66   
Australia                      51.61               45.97   
Austria                          NaN               46.29   
Belgium                        52.03               46.21   
Brazil                           NaN               46.08   

                                                                        
Rank_Level Second Tier Top University Third Tier Top University    All  
country                                                                 
Argentina                         NaN                       NaN  45.66  
Australia                       50.40                     47.47  51.61  
Austria                           NaN                     47.78  47.78  
Belgium                         49.73                     47.14  52.03  
Brazil                          49.82                       NaN  49.82

In [103]:
new_df.unstack().unstack()

country                          Argentina  Australia    Austria    Belgium  \
     Rank_Level                                                               
mean First Tier Top University         NaN  47.942500        NaN  51.875000   
     Other Top Unversity         44.672857  44.645750  44.864286  45.081000   
     Second Tier Top University        NaN  49.242500        NaN  49.084000   
     Third Tier Top University         NaN  47.285000  47.066667  46.746667   
     All                         44.672857  45.825517  45.139583  47.011000   
amax First Tier Top University         NaN  51.610000        NaN  52.030000   
     Other Top Unversity         45.660000  45.970000  46.290000  46.210000   
     Second Tier Top University        NaN  50.400000        NaN  49.730000   
     Third Tier Top University         NaN  47.470000  47.780000  47.140000   
     All                         45.660000  51.610000  47.780000  52.030000   

country                             Brazil  Bulgaria     Canada    Chile  \
     Rank_Level                                                            
mean First Tier Top University         NaN       NaN  53.633846      NaN   
     Other Top Unversity         44.499706    44.335  44.760541  44.7675   
     Second Tier Top University  49.565000       NaN  49.218182      NaN   
     Third Tier Top University         NaN       NaN  46.826364      NaN   
     All                         44.781111    44.335  47.359306  44.7675   
amax First Tier Top University         NaN       NaN  60.870000      NaN   
     Other Top Unversity         46.080000    44.480  45.740000  45.3300   
     Second Tier Top University  49.820000       NaN  51.230000      NaN   
     Third Tier Top University         NaN       NaN  47.690000      NaN   
     All                         49.820000    44.480  60.870000  45.3300   

country                              China  Colombia  ...  Switzerland  \
     Rank_Level                                       ...                
mean First Tier Top University   53.592500       NaN  ...    54.005000   
     Other Top Unversity         44.564267   44.4325  ...    44.625000   
     Second Tier Top University  47.868000       NaN  ...    48.184000   
     Third Tier Top University   46.926250       NaN  ...    47.930000   
     All                         44.992575   44.4325  ...    51.208846   
amax First Tier Top University   55.300000       NaN  ...    72.180000   
     Other Top Unversity         45.920000   44.8500  ...    45.050000   
     Second Tier Top University  48.140000       NaN  ...    49.170000   
     Third Tier Top University   47.760000       NaN  ...    47.930000   
     All                         55.300000   44.8500  ...    72.180000   

country                             Taiwan   Thailand  Turkey         USA  \
     Rank_Level                                                             
mean First Tier Top University   54.210000        NaN     NaN   61.066726   
     Other Top Unversity         44.476667  44.830000  44.481   44.871718   
     Second Tier Top University        NaN        NaN     NaN   49.069524   
     Third Tier Top University   47.065000  46.550000     NaN   46.818333   
     All                         45.012391  45.116667  44.481   51.839860   
amax First Tier Top University   54.230000        NaN     NaN  100.000000   
     Other Top Unversity         45.490000  45.860000  45.570   46.300000   
     Second Tier Top University        NaN        NaN     NaN   51.270000   
     Third Tier Top University   47.160000  46.550000     NaN   47.820000   
     All                         54.230000  46.550000  45.570  100.000000   

country                          Uganda  United Arab Emirates  United Kingdom  \
     Rank_Level                                                                 
mean First Tier Top University      NaN                   NaN       63.937931   
     Other Top Unversity          44.28                 44.22       44.881299   


In [ ]:
# Utilizando a função 'stack', iremos mover o cabeçalho mais baixo (rank) para ser o indice mais interno.
new_df=new_df.stack()
new_df.head()

In [ ]:
# Utilizando a função 'unstack', iremos mover o indice mais interno(rank) para o cabeçalho mais interno
new_df=new_df.unstack()
new_df.head()

# Date/Time Functionality

-Timestamp

-Period

-DatetimeIndex

-PeriodIndex

### Timestamp

In [ ]:
# Representa uma marca temporal, associada com um único ponto no tempo.

# Por exemplo, vamos criar um timestamp usando a string 9/1/2019 10:05AM.
pd.Timestamp('9/1/2019 10:05AM')

In [ ]:
# Podemos criar um timestamp passando multíplos parâmetros, como ano,mês,data,hora e minutos, separadamente.
pd.Timestamp(2019,12,20,0,0)

In [ ]:
# O atributo 'isoweekday()' retorna o dia da semana de um timestamp. 1 representa "Monday" e 7 "Sunday".
pd.Timestamp(2019, 12, 20, 0, 0).isoweekday()

In [ ]:
# Podemos extrair o ano, mês, dia, hora, minuto e segundo de um timestamp
print(pd.Timestamp(2019, 12, 20, 5, 2,23).year)
print(pd.Timestamp(2019, 12, 20, 5, 2,23).month)
print(pd.Timestamp(2019, 12, 20, 5, 2,23).day)
print(pd.Timestamp(2019, 12, 20, 5, 2,23).hour)
print(pd.Timestamp(2019, 12, 20, 5, 2,23).minute)
print(pd.Timestamp(2019, 12, 20, 5, 2,23).second)


### Period

In [ ]:
# Supondo que não estamos interessados em um ponto do tempo, mas em um intervalo de tempo. 
# Periodo representa um único intervalo de tempo, como um dia ou um mês

# Podemos criar o período representando o mês Janeiro do ano de 2016.
pd.Period('1/2016')

In [ ]:
# Podemos ver que a 'classe' do período é retornada também, nesse caso 'M'(month)

#Outro exemplo:
pd.Period('3/5/2016')

In [ ]:
# Podemos aplicar operações aritméticas (baseadas na 'classe'): como temos "M", então queremos dizer +15 meses
pd.Period('1/2016') + 15

In [ ]:
# Outro exemplo
pd.Period('3/5/2016') - 2

### DatetimeIndex and PeriodIndex

In [ ]:
# O índice de um timestamp é chamado 'DatetimeIndex'.
# Vamos ver um exemplo:

# Vamos criar uma série t1, com indices iguais a timestamp
t1 = pd.Series(list('abc'), [pd.Timestamp('2016-09-01'), pd.Timestamp('2016-09-02'), 
                             pd.Timestamp('2016-09-03')])
t1

In [ ]:
type(t1.index) # DatetimeIndex

In [ ]:
# Similarmente para o período
t2 = pd.Series(list('def'), [pd.Period('2016-09'), pd.Period('2016-10'), 
                             pd.Period('2016-11')])
t2

In [ ]:
type(t2.index)

### Converting to Datetime

In [ ]:
# Vamos supor que tenhamos uma lista de datas no formato string e queremos criar um DataFrame

#Vamos utilizar diferentes formatos como exemplo.
d1 = ['2 June 2013', 'Aug 29, 2014', '2015-06-26', '7/12/16']

ts3 = pd.DataFrame(np.random.randint(10, 100, (4,2)), index=d1, 
                   columns=list('ab'))
ts3

In [ ]:
# Utilizando 'to_datetime', pandas irá tentar converter os valores para Datetime.

ts3.index = pd.to_datetime(ts3.index)
ts3

In [ ]:
# 'to_datetime' tbm tem atributos, podemos passar 'dayfirst=True' para formatar no tipo europeu.
pd.to_datetime('4.7.12', dayfirst=True)

### Timedelta


In [ ]:
# Timedeltas são diferenças de tempo.
# Se quisermos a diferença entre 3 de Setembro e 1 de Setembro, pegaremos um Timedelta de 2 dias.
pd.Timestamp('9/3/2016')-pd.Timestamp('9/1/2016')

In [ ]:
# Podemos também querer saber qual data e tempo será daqui 12 dias e 3 horas a partir do dia 2 de Setembro.
pd.Timestamp('9/2/2016 8:10AM') + pd.Timedelta('12D 3H')

### Offset

In [ ]:
# Offset é similar ao timedelta, mas segue regras específicas do calendário.
# Exemplo: dia útil, fim do mês.

# Vamos criar um timestamp e ver qual dia da semana é
pd.Timestamp('9/4/2016').weekday()

In [ ]:
# Também podemos usar o timestamp com uma semana a frente.
pd.Timestamp('9/4/2016') + pd.offsets.Week()

In [ ]:
# Podemos ir para o final do mês
pd.Timestamp('9/4/2016') + pd.offsets.MonthEnd()

### Working with Dates in a Dataframe

In [ ]:
# Vamos supor que queremos para 9 medidas: 'domingo de 2 em 2 semanas' começando em Outubro/2016.
# Usando o 'date_range', podemos criar esse DatetimeIndex.
# No 'date_range', precisamos especificar a data de inicio ou a data final. Então especificar quantos períodos queremos e a frequência.
dates = pd.date_range('10-01-2016', periods=9, freq='2W-SUN')
dates

In [ ]:
# Existem varios jeitos de especificar uma frequência, por exemplo o dia útil 'business day'
pd.date_range('10-01-2016', periods=9, freq='B')

In [ ]:
# Ou 1 quarto do ano = 3 meses
pd.date_range('04-01-2016', periods=12, freq='QS-JUN')

In [ ]:
# Voltando ao 'dates'. Vamos criar um dataframe utilizando essas datas
dates = pd.date_range('10-01-2016', periods=9, freq='2W-SUN')
df = pd.DataFrame({'Count 1': 100 + np.random.randint(-5, 10, 9).cumsum(),
                  'Count 2': 120 + np.random.randint(-5, 10, 9)}, index=dates)
df

In [ ]:
# Primeiro, podemos checar qual dia da semana para verificar.
df.index.weekday

In [ ]:
# Podemos usar a função diff() para encontrar a diferença entre os valores: diff() calcula a diferença na célula atual com a mais acima
df.diff()

In [ ]:
# Vamos supor que queremos saber a média de cada mês. Podemos fazer isso utilizando o 'resample' que agrupa os indices pelo o que especificarmos.
df.resample('M').mean()

In [ ]:
# Podemos pegar as colunas que satisfazem um ano por exemplo:
df.loc['2017']

In [ ]:
# Ou pegar um mês em particular
df.loc['2016-12']

In [ ]:
# Podemos pegar uma determinada para frente
df.loc['2016-12':]